In [1]:
import time
from sklearn.random_projection import SparseRandomProjection, _sparse_random_matrix, johnson_lindenstrauss_min_dim

In [9]:
projection_matrix = _sparse_random_matrix(n_components=1000, n_features=500000, density="auto", random_state=42)

In [13]:
n_components_ = johnson_lindenstrauss_min_dim(n_samples=1000, eps=0.1)

In [ ]:
n_components_ = johnson_lindenstrauss_min_dim(n_samples=1000, eps=0.1) # this step is efficient. 
projection_matrix = _sparse_random_matrix(n_components=n_components_, n_features=500000, density="auto", random_state=42) # this step is not efficient! in the old implementation, it uses a dense matrix to generate the projection matrix.

In [12]:
projection_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4185511 stored elements and shape (5920, 500000)>

In [6]:
import numpy as np
import scipy.sparse as sp
import torch
from math import sqrt

# --- Original numpy/scipy version for reference ---
def _check_input_size(n_components, n_features):
    if n_components <= 0 or n_features <= 0:
        raise ValueError("n_components and n_features must be > 0.")

def _check_density(density, n_features):
    if density == "auto":
        density = 1 / sqrt(n_features)
    if not (0 < density <= 1):
        raise ValueError("density must be in (0, 1].")
    return density

def sample_without_replacement(n, k, random_state):
    # a simple implementation using np.random.choice with replace=False
    return np.random.choice(n, size=k, replace=False)

def _sparse_random_matrix(n_components, n_features, density="auto", random_state=None):
    """Original numpy/scipy version."""
    _check_input_size(n_components, n_features)
    density = _check_density(density, n_features)
    rng = np.random.RandomState(random_state)

    if density == 1:
        components = rng.binomial(1, 0.5, (n_components, n_features)) * 2 - 1
        return 1 / np.sqrt(n_components) * components
    else:
        indices = []
        offset = 0
        indptr = [offset]
        for _ in range(n_components):
            n_nonzero_i = rng.binomial(n_features, density)
            indices_i = sample_without_replacement(n_features, n_nonzero_i, random_state=rng)
            indices.append(indices_i)
            offset += n_nonzero_i
            indptr.append(offset)

        indices = np.concatenate(indices) if indices else np.array([], dtype=int)
        data = rng.binomial(1, 0.5, size=np.size(indices)) * 2 - 1

        components = sp.csr_matrix(
            (data, indices, indptr), shape=(n_components, n_features)
        )

        return np.sqrt(1 / density) / np.sqrt(n_components) * components

# --- Torch version  ---
def _sparse_random_matrix_torch(n_components, n_features, density="auto", random_state=None, device="cpu", use_multinomial=False):
    """
    Torch version of the sparse random matrix generator.
    Returns a torch tensor: dense if density==1, otherwise a sparse_coo_tensor.
    """
    if density == "auto":
        density = 1 / sqrt(n_features)
    if not (0 < density <= 1):
        raise ValueError("density must be in (0, 1].")
    
    # Setup a torch generator to have reproducible results
    gen = torch.Generator(device=device if device != "mps" else "cpu")
    if random_state is not None:
        gen.manual_seed(random_state)
    
    if density == 1:
        # Dense: each entry is randomly ±1
        components = torch.randint(0, 2, (n_components, n_features), 
                                 generator=gen, device=device)
        components = components * 2 - 1
        return components / sqrt(n_components)
    else:
        indices_list = []
        values_list = []
        # For each row, sample the number of nonzeros and choose indices without replacement
        for i in range(n_components):
            # Draw number of nonzero entries via a Binomial distribution.
            # We use torch.distributions.Binomial to sample
            binom = torch.distributions.Binomial(total_count=n_features, probs=density)
            n_nonzero = int(binom.sample().item())
            feature_weights = torch.ones(n_features, device=device) # / n_features
            if n_nonzero > 0:
                # Choose without replacement: use randperm and take the first n_nonzero entries
                # row_indices = torch.randperm(n_features, generator=gen, device=device)[:n_nonzero]
                if use_multinomial:
                    row_indices = torch.multinomial(feature_weights, n_nonzero, replacement=False)
                else:
                    row_indices = torch.randperm(n_features, generator=gen, device=device)[:n_nonzero]
                # Sorting the indices (optional)
                row_indices, _ = torch.sort(row_indices)
                # Append the row index repeated for each nonzero
                row_ids = torch.full((n_nonzero,), i, dtype=torch.long, device=device)
                indices_list.append(torch.stack([row_ids, row_indices], dim=0))
                # Random ±1 values
                values = torch.randint(0, 2, (n_nonzero,), generator=gen, device=device) * 2 - 1
                values_list.append(values.to(torch.float32))
        
        if indices_list:
            indices = torch.cat(indices_list, dim=1)
            values = torch.cat(values_list)
        else:
            indices = torch.empty((2, 0), dtype=torch.long, device=device)
            values = torch.tensor([], dtype=torch.float32, device=device)
        
        # Scale factor as in the numpy version
        scale = sqrt(1 / density) / sqrt(n_components)
        values = values * scale  # Scale the values before creating the tensor
        
        # Create the sparse tensor with scaled values
        sparse_tensor = torch.sparse_coo_tensor(indices, values, 
                                              size=(n_components, n_features),
                                              device=device)
        return sparse_tensor


### Implementation of random index selection through multinomial vs permutation

In [8]:
import time
n_components = 5000
n_features = 500000
density = "auto"
random_state = 42
# torch_matrix = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cpu")
t0 = time.time()
torch_matrix_cuda = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cuda", use_multinomial=True)
torch_cuda_time = time.time() - t0
print(f"Torch matrix on GPU generation time: {torch_cuda_time:.4f} seconds")

t0 = time.time()
torch_matrix_cuda = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cuda", use_multinomial=False)
torch_cuda_time = time.time() - t0
print(f"Torch matrix on GPU generation time: {torch_cuda_time:.4f} seconds with use_multinomial=False")

t0 = time.time()
torch_matrix_cuda = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cpu", use_multinomial=True)
torch_cuda_time = time.time() - t0
print(f"Torch matrix on CPU generation time: {torch_cuda_time:.4f} seconds")

t0 = time.time()
torch_matrix_cuda = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cpu", use_multinomial=False)
torch_cuda_time = time.time() - t0
print(f"Torch matrix on CPU generation time: {torch_cuda_time:.4f} seconds with use_multinomial=False")

Torch matrix on GPU generation time: 2.1220 seconds
Torch matrix on GPU generation time: 1.6251 seconds with use_multinomial=False
Torch matrix on CPU generation time: 16.0217 seconds
Torch matrix on CPU generation time: 19.9218 seconds with use_multinomial=False


In [19]:
def benchmark_sparse_random_projection(n_samples = 1000,
                                        n_components = "auto",
                                        n_features = 500000,
                                        density = "auto",
                                        random_state = 42):
    if n_components == "auto":
        n_components = johnson_lindenstrauss_min_dim(n_samples, eps=0.1)
    # Time numpy version
    t0 = time.time()
    np_matrix = _sparse_random_matrix(n_components, n_features, density=density, random_state=random_state)
    np_time = time.time() - t0
    print(f"Numpy matrix generation time: {np_time:.4f} seconds")
    # print the size and non-zero ratio of the numpy matrix
    print(f"Numpy matrix size: {np_matrix.shape}")
    print(f"Numpy matrix non-zero ratio: {np_matrix.nnz/np_matrix.shape[0]/np_matrix.shape[1]:.4f}")
    # Time torch version 
    t0 = time.time()
    torch_matrix = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cpu")
    torch_time = time.time() - t0
    print(f"Torch matrix on CPU generation time: {torch_time:.4f} seconds")
    # print the size and non-zero ratio of the torch matrix
    print(f"Torch matrix size: {torch_matrix.shape}")
    print(f"Torch matrix non-zero ratio: {torch_matrix._nnz()/torch_matrix.shape[0]/torch_matrix.shape[1]:.4f}")
    # Time torch version on GPU
    t0 = time.time()
    torch_matrix_cuda = _sparse_random_matrix_torch(n_components, n_features, density=density, random_state=random_state, device="cuda")
    torch_cuda_time = time.time() - t0
    print(f"Torch matrix on GPU generation time: {torch_cuda_time:.4f} seconds")
    # print the size and non-zero ratio of the torch matrix
    print(f"Torch matrix size: {torch_matrix_cuda.shape}")
    print(f"Torch matrix non-zero ratio: {torch_matrix_cuda._nnz()/torch_matrix_cuda.shape[0]/torch_matrix_cuda.shape[1]:.4f}")
    print(f"Speedup factor: {np_time/torch_cuda_time:.2f}x")
    
    
    # benchmark the time it takes to matmul a dense random matrix
    featmat = torch.randn(n_samples, n_features,)
    # benchmark the time it takes to matmul with numpy
    t0 = time.time()
    featmat.numpy() @ np_matrix.T
    np_time = time.time() - t0
    print(f"Numpy matrix matmul time: {np_time:.4f} seconds")
    
    # benchmark the time it takes to matmul with torch matrix on CPU
    t0 = time.time()
    featmat @ torch_matrix.T
    torch_cpu_time = time.time() - t0
    print(f"Torch matrix on CPU matmul time: {torch_cpu_time:.4f} seconds")
    t0 = time.time()
    torch.sparse.mm(featmat, torch_matrix.T)
    torch_cpu_time = time.time() - t0
    print(f"Torch matrix on CPU sparse matmul time: {torch_cpu_time:.4f} seconds")
    
    # benchmark the time it takes to matmul with torch matrix on GPU
    t0 = time.time()
    featmat.to("cuda") @ torch_matrix_cuda.T
    torch_cuda_time = time.time() - t0
    print(f"Torch matrix on GPU matmul time: {torch_cuda_time:.4f} seconds")
    t0 = time.time()
    torch.sparse.mm(featmat.to("cuda"), torch_matrix_cuda.T)
    torch_cuda_time = time.time() - t0
    print(f"Torch matrix on GPU sparse matmul time: {torch_cuda_time:.4f} seconds")
    print(f"Speedup factor: {np_time/torch_cuda_time:.2f}x")
    return np_matrix, torch_matrix, torch_matrix_cuda

In [17]:
benchmark_sparse_random_projection(n_samples = 1000,
                                   n_components = "auto",
                                   n_features = 50000,
                                   density = "auto",
                                   random_state = 42);

Numpy matrix generation time: 4.6529 seconds
Numpy matrix size: (5920, 50000)
Numpy matrix non-zero ratio: 0.0045
Torch matrix on CPU generation time: 2.8734 seconds
Torch matrix size: torch.Size([5920, 50000])
Torch matrix non-zero ratio: 0.0045
Torch matrix on GPU generation time: 1.8920 seconds
Torch matrix size: torch.Size([5920, 50000])
Torch matrix non-zero ratio: 0.0045
Speedup factor: 2.46x
Numpy matrix matmul time: 1.1698 seconds
Torch matrix on CPU matmul time: 7.2082 seconds
Torch matrix on GPU matmul time: 0.0211 seconds
Speedup factor: 55.35x


In [18]:
benchmark_sparse_random_projection(n_samples = 1000,
                                   n_components = "auto",
                                   n_features = 500000,
                                   density = "auto",
                                   random_state = 42);

Numpy matrix generation time: 47.9755 seconds
Numpy matrix size: (5920, 500000)
Numpy matrix non-zero ratio: 0.0014
Torch matrix on CPU generation time: 23.3475 seconds
Torch matrix size: torch.Size([5920, 500000])
Torch matrix non-zero ratio: 0.0014
Torch matrix on GPU generation time: 1.9327 seconds
Torch matrix size: torch.Size([5920, 500000])
Torch matrix non-zero ratio: 0.0014
Speedup factor: 24.82x
Numpy matrix matmul time: 5.4424 seconds
Torch matrix on CPU matmul time: 35.4209 seconds
Torch matrix on GPU matmul time: 0.2012 seconds
Speedup factor: 27.05x


#### Benchmarking SparseRandomProjection with n_samples=1000, n_features=50000, n_components=auto, eps=0.1, random_state=42
* Sklearn time: 3.5581 seconds
* Torch time: 1.9403 seconds
* Speedup factor: 1.83x

#### Benchmarking SparseRandomProjection with n_samples=1000, n_features=500000, n_components=auto, eps=0.1, random_state=42
* Sklearn time: 12.5077 seconds
* Torch time: 2.5684 seconds
* Speedup factor: 4.87x
--------------------------------
#### Benchmarking sparse matrix generation and matmul with n_samples=1000, n_features=50000, n_components=auto, density=auto, random_state=42
* Numpy matrix generation time: 2.8102 seconds
* Numpy matrix size: (5920, 50000)
* Numpy matrix non-zero ratio: 0.0045
* Torch matrix on CPU generation time: 2.7074 seconds
* Torch matrix size: torch.Size([5920, 50000])
* Torch matrix non-zero ratio: 0.0045
* Torch matrix on GPU generation time: 1.7917 seconds
* Torch matrix size: torch.Size([5920, 50000])
* Torch matrix non-zero ratio: 0.0045
* Speedup factor: 1.57x

* Numpy matrix matmul time: 1.1104 seconds
* Torch matrix on CPU matmul time: 7.0737 seconds
* Torch matrix on CPU sparse matmul time: 7.0749 seconds
* Torch matrix on GPU matmul time: 0.0197 seconds
* Torch matrix on GPU sparse matmul time: 0.0364 seconds
* Speedup factor: 30.48x

#### Benchmarking sparse matrix generation and matmul with n_samples=1000, n_features=500000, n_components=auto, density=auto, random_state=42
* Numpy matrix generation time: 8.8679 seconds
* Numpy matrix size: (5920, 500000)
* Numpy matrix non-zero ratio: 0.0014
* Torch matrix on CPU generation time: 22.1904 seconds
* Torch matrix size: torch.Size([5920, 500000])
* Torch matrix non-zero ratio: 0.0014
* Torch matrix on GPU generation time: 1.8465 seconds
* Torch matrix size: torch.Size([5920, 500000])
* Torch matrix non-zero ratio: 0.0014
* Speedup factor: 4.80x

* Numpy matrix matmul time: 5.2763 seconds
* Torch matrix on CPU matmul time: 34.7338 seconds
* Torch matrix on CPU sparse matmul time: 34.6512 seconds
* Torch matrix on GPU matmul time: 0.1881 seconds
* Torch matrix on GPU sparse matmul time: 0.2777 seconds
* Speedup factor: 19.00x

**Conclusion**
* Generally, generating sparse random matrix is faster through torch cpu than numpy, much faster on CUDA (2x to 24x speedup). 
* However, matrix multiplication in torch cpu is slower (x5,x7 slow down)! Potentially due to non optimized sparse matmul implementation. 
* But matmul with sparse weights on CUDA is much faster 27x to 55x speed up than the numpy sparse matrix multiplication

In [29]:
Xtsr = torch.randn(1000, 500000, device="cuda")
projection_matrix = _sparse_random_matrix_torch(2500, 500000, density="auto", random_state=42, device="cuda", use_multinomial=False)

In [30]:
Xtsr @ projection_matrix.T

torch.sparse.mm(Xtsr, projection_matrix.T)

tensor([[  0.9790,   3.4008,  -9.6979,  ..., -10.0966,  -4.7390,  20.8956],
        [-19.4450, -11.0050,   6.1506,  ...,   6.2979,  -7.8975,   0.6770],
        [-29.1357,  11.3054, -13.5277,  ...,  35.4753,   4.0125,  -7.0463],
        ...,
        [ -6.6439,  33.0906, -14.1929,  ...,  -4.3143,  -2.8030,  -2.1197],
        [  1.1072,   5.0099,  -2.1067,  ...,   2.5069,  16.5518,   1.1054],
        [ -9.8507,  28.1183, -12.2733,  ...,  -9.0177,   2.4313, -17.1498]],
       device='cuda:0')

tensor([[  0.9790,   3.4008,  -9.6979,  ..., -10.0966,  -4.7390,  20.8956],
        [-19.4450, -11.0050,   6.1506,  ...,   6.2979,  -7.8975,   0.6770],
        [-29.1357,  11.3054, -13.5277,  ...,  35.4753,   4.0125,  -7.0463],
        ...,
        [ -6.6439,  33.0906, -14.1929,  ...,  -4.3143,  -2.8030,  -2.1197],
        [  1.1072,   5.0099,  -2.1067,  ...,   2.5069,  16.5518,   1.1054],
        [ -9.8507,  28.1183, -12.2733,  ...,  -9.0177,   2.4313, -17.1498]],
       device='cuda:0')

: 

In [28]:
import torch
import numpy as np
import scipy.sparse as sp
from sklearn.random_projection import SparseRandomProjection

def torch_to_sklearn_projection(torch_sparse, n_components, ):
    # Move to CPU and ensure the tensor is coalesced
    torch_sparse = torch_sparse.cpu().coalesce()
    # Extract indices and values from the torch sparse tensor.
    indices = torch_sparse.indices().numpy()
    values = torch_sparse.values().numpy()
    shape = torch_sparse.size()  # Expected shape: (n_components, n_features)
    # Create a scipy sparse matrix (using COO and converting to CSR for efficiency)
    scipy_random_matrix = sp.coo_matrix((values, (indices[0], indices[1])), shape=shape).tocsr()
    # Initialize a SparseRandomProjection instance.
    srp = SparseRandomProjection(n_components=n_components, dense_output=True)
    # Assign the random projection matrix to the sklearn object.
    srp.components_ = scipy_random_matrix
    return srp


def test_projection_consistency(n_components = 1000, 
                                n_features = 500000, 
                                n_samples = 1000, 
                                density = 0.0014,   
                                random_state = 42):
    # Define dimensions.
    # Adjust density for the sparse projection matrix.
    # Set random seeds for reproducibility.
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    # Create a random sparse projection matrix in torch.
    # Here, we simulate a matrix with values either +0.3761 or -0.3761.
    total_elements = n_components * n_features
    nnz = int(total_elements * density)
    # Randomly pick indices for non-zero entries.
    indices_0 = torch.randint(0, n_components, (nnz,))
    indices_1 = torch.randint(0, n_features, (nnz,))
    indices = torch.stack([indices_0, indices_1], dim=0)
    # Randomly choose values +0.3761 or -0.3761.
    values = torch.empty(nnz).uniform_()  # uniform random values in [0,1)
    values = torch.where(values < 0.5, torch.tensor(-0.3761), torch.tensor(0.3761))
    # Create the sparse tensor.
    torch_sparse = torch.sparse_coo_tensor(indices, values, size=(n_components, n_features))
    # Convert torch sparse matrix to sklearn random projection.
    srp = torch_to_sklearn_projection(torch_sparse, n_components, n_features)
    # Generate a random input data matrix X.
    X_np = np.random.randn(n_samples, n_features).astype(np.float32)
    X_torch = torch.from_numpy(X_np)
    # Compute projection with sklearn: X * components_.T.
    X_proj_sklearn = srp.transform(X_np)
    # Compute projection with torch.
    # Since torch_sparse has shape (n_components, n_features), we need its transpose for X * A.T.
    torch_sparse_transposed = torch_sparse.transpose(0, 1).coalesce()  # shape: (n_features, n_components)
    # Perform sparse-dense matrix multiplication.
    X_proj_torch = torch.sparse.mm(X_torch, torch_sparse_transposed).numpy()  # shape: (n_samples, n_components)
    # Compare results.
    diff = np.abs(X_proj_torch - X_proj_sklearn).max()
    print("Maximum difference between torch and sklearn projections:", diff)
    # Assert that the results are nearly equal.
    assert np.allclose(X_proj_torch, X_proj_sklearn, atol=1e-4), "Results are different!"
    print("Test passed: torch projection matches sklearn projection!")

if __name__ == '__main__':
    test_projection_consistency()

Maximum difference between torch and sklearn projections: 1.7166138e-05
Test passed: torch projection matches sklearn projection!


In [4]:
torch_matrix_cuda

tensor(indices=tensor([[     0,      0,      0,  ...,   4999,   4999,   4999],
                       [   415,    472,    767,  ..., 497710, 499147, 499828]]),
       values=tensor([ 0.3761, -0.3761, -0.3761,  ...,  0.3761,  0.3761,
                       0.3761]),
       device='cuda:0', size=(5000, 500000), nnz=3531808,
       layout=torch.sparse_coo)

In [1]:
import torch
import math
import numpy as np
try:
    from sklearn.random_projection import SparseRandomProjection, _sparse_random_matrix, johnson_lindenstrauss_min_dim
except ImportError:
    raise ImportError("scikit-learn must be installed to export to sklearn.")

class SparseRandomProjection_torch:
    def __init__(self, n_components=None, random_state=None):
        """
        Parameters:
          n_components : int, optional
              The target dimensionality. If None, will be set during fitting (default: int(sqrt(n_features))).
          random_state : int, optional
              Seed for reproducibility.
        """
        self.n_components = n_components
        self.random_state = random_state
        self.projection_matrix = None
        if random_state is not None:
            torch.manual_seed(random_state)

    def fit(self, X):
        """
        Fit the random projection matrix based on input tensor X.
        
        Parameters:
          X : torch.Tensor of shape (n_samples, n_features)
          
        Returns:
          self
        """
        if not isinstance(X, torch.Tensor):
            raise ValueError("Input X must be a torch.Tensor")
        
        n_samples, n_features = X.shape
        # Set default n_components if not provided
        if self.n_components is None:
            self.n_components = int(math.sqrt(n_features))
            if self.n_components < 1:
                self.n_components = 1

        # Determine density and scale as in the sparse random projection literature.
        # Density: probability that any given entry is nonzero.
        density = 1.0 / math.sqrt(n_features)
        # Scaling factor to maintain norm in expectation.
        scale = math.sqrt(1.0 / density)  # equals n_features^(1/4)

        # Create the random matrix
        # Generate a mask with probability "density" for nonzero entries.
        mask = (torch.rand(n_features, self.n_components) < density).float()
        # Generate random signs (+1 or -1) for nonzero entries.
        signs = torch.randint(0, 2, (n_features, self.n_components)).float() * 2 - 1
        # The projection matrix: non-zero entries are ±scale, zeros otherwise.
        self.projection_matrix = mask * signs * scale

        return self

    def transform(self, X):
        """
        Project the data X using the learned projection matrix.
        
        Parameters:
          X : torch.Tensor of shape (n_samples, n_features)
          
        Returns:
          X_projected : torch.Tensor of shape (n_samples, n_components)
        """
        if self.projection_matrix is None:
            raise RuntimeError("The model has not been fitted yet. Call 'fit' or 'fit_transform' first.")
        return torch.matmul(X, self.projection_matrix)

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        
        Parameters:
          X : torch.Tensor of shape (n_samples, n_features)
          
        Returns:
          X_projected : torch.Tensor of shape (n_samples, n_components)
        """
        self.fit(X)
        return self.transform(X)

    def export_to_sklearn(self):
        """
        Exports the current projection matrix to a scikit-learn SparseRandomProjection instance.
        The sklearn object will have its 'components_' attribute set to the projection matrix.
        Note: The sklearn SparseRandomProjection expects components_ of shape (n_components, n_features),
        so the projection matrix is transposed.
        
        Returns:
          skrp : sklearn.random_projection.SparseRandomProjection instance with the learned components.
        """

        # Create an instance with the same n_components.
        skrp = SparseRandomProjection(n_components=self.n_components)
        
        # The sklearn transformer stores components_ as a sparse matrix of shape (n_components, n_features).
        # Convert the torch tensor to a numpy array and then to a scipy sparse matrix.
        import scipy.sparse
        
        # Ensure projection_matrix is on CPU and convert to numpy.
        proj_np = self.projection_matrix.cpu().numpy()
        # Transpose so that shape becomes (n_components, n_features).
        skrp.components_ = scipy.sparse.csc_matrix(proj_np.T)
        return skrp

# Example usage:
if __name__ == '__main__':
    # Create a random feature matrix of shape (n_samples, n_features)
    n_samples, n_features = 100, 50
    featmat = torch.randn(n_samples, n_features)
    
    # Using a specified n_components
    n_components = 10
    srp_transformer = SparseRandomProjection_torch(n_components=n_components, random_state=42)
    featmat_srp = srp_transformer.fit_transform(featmat)
    print("Projected feature matrix shape with specified n_components:", featmat_srp.shape)
    
    # Using default n_components (automatically determined)
    srp_transformer_default = SparseRandomProjection_torch(random_state=42)
    featmat_srp_default = srp_transformer_default.fit_transform(featmat)
    print("Projected feature matrix shape with default n_components:", featmat_srp_default.shape)
    
    # Exporting to sklearn version:
    sk_srp = srp_transformer.export_to_sklearn()
    print("Exported sklearn components_ shape:", sk_srp.components_.shape)

Projected feature matrix shape with specified n_components: torch.Size([100, 10])
Projected feature matrix shape with default n_components: torch.Size([100, 7])
Exported sklearn components_ shape: (10, 50)


In [4]:
def test_projection_consistency():
    """
    Test function to compare the PyTorch-based SparseRandomProjection with scikit-learn's implementation.
    
    It fits the torch-based transformer, exports its projection matrix to a sklearn object,
    transforms the data using both methods, and asserts that the outputs are equal (within tolerance).
    """
    # Fix random seed for reproducibility.
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Generate a random feature matrix.
    n_samples, n_features = 100, 50
    X_torch = torch.randn(n_samples, n_features)
    
    # Instantiate and fit the torch-based transformer.
    n_components = 10
    srp_torch = SparseRandomProjection_torch(n_components=n_components, random_state=42)
    X_proj_torch = srp_torch.fit_transform(X_torch)
    
    srp_sklearn = SparseRandomProjection(n_components=n_components, random_state=42)
    X_proj_sklearn = srp_sklearn.fit_transform(X_torch.numpy())
    
    # Export to scikit-learn transformer.
    # srp_sklearn = srp_torch.export_to_sklearn()
    
    # Transform the same data using the sklearn transformer.
    # X_np = X_torch.numpy()
    # X_proj_sklearn = srp_sklearn.transform(X_np)
    
    # Compute the maximum absolute difference between the two results.
    diff = np.abs(X_proj_torch.numpy() - X_proj_sklearn).max()
    print("Maximum difference between torch and sklearn projections:", diff)
    
    # Assert that the results are nearly equal.
    assert np.allclose(X_proj_torch.numpy(), X_proj_sklearn, atol=1e-6), "Results are different!"
    print("Test passed: torch projection matches sklearn projection!")

test_projection_consistency()

Maximum difference between torch and sklearn projections: 23.561712


AssertionError: Results are different!